## Simulation of fields penetrating two slabs with oblique incidence
### Colin Renard (50012000) & Mathieu Reniers (30322000)

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from matplotlib import cm
from numba import jit
from tqdm.autonotebook import trange, tqdm
%matplotlib qt


C:\Users\mathi\AppData\Local\Temp\ipykernel_16288\3757444917.py:6: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange, tqdm


#### Parameters

In [3]:
Nf = 50  #250
fc = 3e9         
df = 0.5e9                              # Deviation frequency [Hz]
B = (fc+df) - (fc-df)                   # Bandwidth [Hz]
f = np.linspace(fc-df,fc+df,Nf)         # Frequency axis [Hz]
w,wc = 2*np.pi*f, 2*np.pi*fc            # Pulsation axis & central pulsation
sigma_w = 2*np.pi*0.2*df                # Standard deviation of pulsations

A = (1/(np.sqrt(2*np.pi)*sigma_w)) * np.exp(-(w-wc)**2/(2*(sigma_w**2)))   # Amplitude distribution over frequencies (Gaussian)
#A = A*np.sqrt(4*sigma_w*(np.pi**(3/2)))                                   # Normalization (using Parseval relation, see Homework1)
#plt.plot(f,A)

# Vacuum
n0 = 1; e0 = 8.85e-12; mu0 = 4*np.pi*1e-7
c = 1/np.sqrt(e0*mu0)

# Media

a = 0.6                 # Scaling factor

l0 = 2*a                # Length of medium 0 [m]
k0 = w * n0 /c          # Wavenumber 0

n1 = 1.2                # Refractive index of medium 1
l1 = 2.5*a              # Length of medium 1 [m]
k1 = k0 * n1            # Wavenumber 1

n2 = 1.7                # Refractive index of medium 2
l2 = 2.5*a              # Length of medium 2 [m]
k2 = k0 * n2            # Wavenumber 2

n3 = n0                 # Refractive index of medium 3
l3 = 2*a                # Length of medium 3 [m]
k3 = k0 * n3            # Wavenumber 3

# Axis
Nz = 200           # Precision
z0 = np.linspace(0,l0, int(l0*Nz))
z1 = np.linspace(0,l1, int(l1*Nz))
z2 = np.linspace(0,l2, int(l2*Nz))
z3 = np.linspace(0,l3, int(l3*Nz))

z = np.array([*z0, *z1 + l0, *z2 + (l0+l1), *z3 + (l0+l1+l2)])

# 2D plot
Nx = 200 #20
x = np.linspace(0,3,Nx)
X_mesh, Z_mesh = np.meshgrid(x, z)


#### Useful functions and associated parameters

In [4]:
rho     =   lambda n,np : (n-np)/(n+np)                                 # Return rho of interface n|np
tau     =   lambda n,np : (2*n)/(n+np)                                  # Return tau of interface n|np
gamma   =   lambda gammap, rho : (rho + gammap) / (1 + rho*gammap)      # Return gamma of interface rho: _ | gammap

# Compute gamma in the same media at next interface (R -> L)
def prev_gamma(gamma, k, l):
    gammap = gamma*np.exp(-2*1j*k*l)
    return gammap

deg2rad = lambda deg : (deg*np.pi)/180
rad2deg = lambda rad : (rad*180)/np.pi

# Computation of theta' angle with Snell's law
def next_theta(theta, n, nprim):
    thetap = np.arcsin((n/nprim)*np.sin(theta))
    return thetap

theta1 = deg2rad(40)                          # Angle of incidence [°] - left of interface I  -> media 0
# theta1p = next_theta(theta1,n0,n1)          # Right of interface I = Left of interface II   -> media 1
# theta2p = next_theta(theta1p,n1,n2)         # Right of interface II = Left of interface III -> media 2
# theta3p = next_theta(theta2p,n2,n3)         # Right of interface III                        -> media 3


# Alternative to Snell's Law -> use wavenumer vectors and continuity of fields

kx0 = k0*np.sin(theta1)             # kx_+ = kx_-
kz0 = k0*np.cos(theta1)             # kz_+ 

kx1 = kx0                           
kz1 = np.sqrt(k1**2 - kx1**2)

kx2 = kx1
kz2 = np.sqrt(k2**2 - kx2**2)

kx3 = kx2
kz3 = np.sqrt(k3**2 - kx3**2)


# TM polarization
n0_T = n0/(kz0/k0)[0]
n1_T = n1/(kz1/k1)[0]
n2_T = n2/(kz2/k2)[0]
n3_T = n3/(kz3/k3)[0]

# Computation of gamma /TRANSVERSE/ (1D array for each frequency)
gamma_3 = gamma(0,rho(n2_T,n3_T))
gamma_2p = prev_gamma(gamma_3,kz2,l2)
gamma_2 = gamma(gamma_2p,rho(n1_T,n2_T))
gamma_1p = prev_gamma(gamma_2,kz1,l1)
gamma_1 = gamma(gamma_1p,rho(n0_T,n1_T))
gamma_start = prev_gamma(gamma_1,kz0,l0)

def interface_L2R(Ep,Em,n,np):
    rho_ip = rho(np,n)                              # /!\ Special definitions in Orfanidis
    tau_ip = tau(np,n)                              # /!\ Special definitions in Orfanidis
    Ep_ = (1/tau_ip) * (1 * Ep + rho_ip * Em)
    Em_ = (1/tau_ip) * (rho_ip * Ep + 1 * Em)
    return Ep_,Em_

def propagation_L2R(Ep1,Em1,k,l):
    Ep2 = np.exp(-1j*k*l) * Ep1
    Em2 = np.exp(1j*k*l) * Em1
    return Ep2,Em2


#### Computation

In [5]:
Nt = 200
t = np.linspace(0,200*1/fc,Nt)

E = np.zeros(len(z))
E_time = np.zeros((len(t),len(E)))
E_2D = np.zeros((len(t),len(x),len(z)))

for i in tqdm(range(len(t)), "Time instant"):

    # Superposition principle
    for j in range(len(f)):

        E0p_start = 1
        E0m_start = 1*gamma_start[j]
       
        E0p, E0m = propagation_L2R(E0p_start, E0m_start, kz0[j], z0)        # Waves in (0)

        E1p_l, E1m_l = interface_L2R(E0p[-1],E0m[-1],n0_T,n1_T)             # Right value at interface I
        E1p, E1m = propagation_L2R(E1p_l, E1m_l, kz1[j], z1)                # Waves in (1)

        E2p_l, E2m_l = interface_L2R(E1p[-1],E1m[-1],n1_T,n2_T)             # Right value at interface II
        E2p, E2m = propagation_L2R(E2p_l, E2m_l, kz2[j], z2)                # Waves in (2)

        E3p_l, E3m_l = interface_L2R(E2p[-1],E2m[-1],n2_T,n3_T)             # Right value at interface III
        E3p, E3m = propagation_L2R(E3p_l, E3m_l, kz3[j],z3)                 # Waves in (3)

        E = np.array([*(E0p+E0m)*np.exp(-1j*kx0[j]*0), *(E1p+E1m)*np.exp(-1j*kx1[j]*0) , *(E2p+E2m)*np.exp(-1j*kx2[j]*0) , *(E3p+E3m)*np.exp(-1j*kx3[j]*0)])

        E_2D_ampl = np.zeros((len(x),len(z)),dtype="complex")

        for l in range(len(x)):
            E0_tmp = (E0p+E0m)*np.exp(-1j*kx0[j]*x[l])
            E1_tmp = (E1p+E1m)*np.exp(-1j*kx1[j]*x[l])
            E2_tmp = (E2p+E2m)*np.exp(-1j*kx2[j]*x[l])
            E3_tmp = (E3p+E3m)*np.exp(-1j*kx3[j]*x[l])
            E_2D_ampl[l] = np.array([*E0_tmp, *E1_tmp , *E2_tmp , *E3_tmp])
    
        E_time[i] += A[j]*np.real(E*np.exp(1j*w[j]*t[i]))
        E_2D[i] += A[j]*np.real(E_2D_ampl*np.exp(1j*w[j]*t[i]))

E_time /= np.max(E_time)
E_2D /= np.max(E_2D)

Time instant:   0%|          | 0/200 [00:00<?, ?it/s]

#### Plot at a given instant

In [6]:
def plot_instant(i, type):
    if type == "1D":
        c = ["tab:orange", "tab:green", "tab:red", "tab:blue"]
        fig, ax = plt.subplots()

        line, = ax.plot(z, E_time[i])

        yrange = plt.gca().get_ylim()

        plt.fill_between(z0, yrange[0], yrange[1], color=c[1],alpha=0.3, label=r"$n=1$")
        plt.fill_between(l0+z1, yrange[0], yrange[1], color=c[0],alpha=0.3, label=r"$n=$"+str(n1))
        plt.fill_between(l0+l1+z2, yrange[0], yrange[1], color=c[2],alpha=0.3, label=r"$n=$"+str(n2))
        plt.fill_between(l0+l1+l2+z3, yrange[0], yrange[1], color=c[1],alpha=0.3)
        plt.show()
    elif type == "2D":
        fig, ax = plt.subplots()

        cmap = cm.seismic

        surf = ax.imshow(E_2D[i], cmap=cmap, origin='lower',
                 extent=[Z_mesh.min(), Z_mesh.max(), X_mesh.min(), X_mesh.max()])

        fig.colorbar(surf, shrink=0.5, aspect=5)

        ax.axvline(x=l0, color='black', linestyle='--')
        ax.axvline(x=l0+l1, color='black', linestyle='--')
        ax.axvline(x=l0+l1+l2, color='black', linestyle='--')

        plt.show()
    elif type == "3D":
        fig, ax = plt.subplots(subplot_kw={"projection": "3d"})


        surf = ax.plot_surface(X_mesh, Z_mesh, np.transpose(E_2D[i]), cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)
        
        ax.set_xlabel("x")
        ax.set_ylabel("z")

        fig.colorbar(surf, shrink=0.5, aspect=5)

plot_instant(0,"2D")

#### Animation

In [7]:
c = ["tab:orange", "tab:green", "tab:red", "tab:blue"]

fig, ax = plt.subplots()

ax.set_xlabel("z [m]")
ax.set_ylabel("Electric Field [V/m]")
ax.grid()

line, = ax.plot(z, E_time[0])
yrange = [-1,1] # plt.gca().get_ylim()


plt.fill_between(z0, yrange[0], yrange[1], color=c[1],alpha=0.3, label=r"$n=1$")
plt.fill_between(l0+z1, yrange[0], yrange[1], color=c[0],alpha=0.3, label=r"$n=$"+str(n1))
plt.fill_between(l0+l1+z2, yrange[0], yrange[1], color=c[2],alpha=0.3, label=r"$n=$"+str(n2))
plt.fill_between(l0+l1+l2+z3, yrange[0], yrange[1], color=c[1],alpha=0.3)
ax.legend(loc="upper right")

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    line.set_ydata(E_time[i])
    return line,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=100, repeat=True ,blit=False)
plt.show()

#writer = animation.PillowWriter(fps=10, metadata=dict(artist='Me'),bitrate=1800)
#ani.save('oblique_incidence_60°.gif', writer=writer)

In [10]:
c = ["tab:orange", "tab:green", "tab:red", "tab:blue"]


fig, ax = plt.subplots()

ax.set_xlabel("z [m]")
ax.set_ylabel("x [m]")
ax.set_title("Transverse Electric Field")
ax.grid()
cmap = cm.seismic


surf = ax.imshow(E_2D[0], cmap=cmap, origin='lower',extent=[Z_mesh.min(), Z_mesh.max(), X_mesh.min(), X_mesh.max()])

fig.colorbar(surf, shrink=0.5, aspect=5)

ax.axvline(x=l0, color='black', linestyle='--')
ax.axvline(x=l0+l1, color='black', linestyle='--')
ax.axvline(x=l0+l1+l2, color='black', linestyle='--')




#ax.legend(loc="upper right")

def anim(i):       
    #line.set_ydata(compute_s(z,t[i]))  # update the data
    surf.set_array(E_2D[i])
    return surf,

ani = animation.FuncAnimation(fig, anim, frames=len(t),interval=100, repeat=True ,blit=False)
plt.show()
#name = 'layer_oblique_40_2D.gif'
#writer = animation.PillowWriter(fps=10,bitrate=5000)
#ani.save(name, writer=writer,dpi=500)
